# Basic Graph Construction

# Define nodes (functions or chains)

In **LangGraph**, a **node** is a core building block of the graph. Each **node** represents a **step or unit of computation**—it can be a function, a LangChain chain, or even another model call.



### 🧠 What is a Node in LangGraph?

A **node** is:
- A **Python function** (synchronous or asynchronous),
- Or a **LangChain Chain**, Tool, or Runnable,
- That takes the **state** as input and returns an **updated state**.

Each node performs one **logical operation** in your workflow.



### ✅ Characteristics of a Node

| Property       | Description |
|----------------|-------------|
| Input          | A `state` object (usually a dict or Pydantic model) |
| Output         | Modified or enriched `state` |
| Role           | Performs one logical step (e.g., tool calling, reasoning, classification) |
| Types Allowed  | Python function, LangChain chain, or any LangChain-compatible Runnable |



### 🔧 Basic Example — Function as Node

```python
def add_numbers(state: dict) -> dict:
    a = state["a"]
    b = state["b"]
    result = a + b
    state["result"] = result
    return state
```

Usage in a graph:
```python
graph.add_node("adder", add_numbers)
```



### ⚙️ Using LangChain Chain as Node

If you have a LangChain chain (like `LLMChain`), you can plug it in:

```python
from langchain.chains import LLMChain

llm_chain = LLMChain(llm=openai, prompt=some_prompt)

graph.add_node("llm_reasoning", llm_chain)
```

LangGraph will automatically call the chain with state input.



### 🔁 Async Support

You can also define **async functions**:

```python
async def ask_model(state: dict) -> dict:
    query = state["query"]
    result = await openai.acall(query)
    state["response"] = result
    return state
```



### 💡 Best Practices

- Each node should:
  - Take a `state` object
  - Avoid side effects if possible
  - Return an updated state
- Keep logic **focused** per node (like single responsibility)
- Use descriptive node names like `"classifier"`, `"llm_answerer"`, `"math_solver"`



### 📌 Summary

| Concept      | Description |
|--------------|-------------|
| Node Type    | Function or chain |
| Input        | State (dict or structured object) |
| Output       | Updated state |
| Purpose      | One unit of logic in the graph |


#  Define edges (transitions)?

In **LangGraph**, **edges** (also called **transitions**) define how the **state flows** from one **node** to the next.



### 🧠 What is an Edge (Transition)?

An **edge** in LangGraph connects two **nodes** and determines **which node to run next**, based on:

* **Static logic** (always go to node B after A)
* or **dynamic logic** (branch based on the state)



### 🧩 How to Define Edges

You define edges using:

```python
graph.add_edge("node_a", "node_b")
```

This means:

> After `node_a` finishes, go to `node_b`.



### 🔀 Conditional Transitions (Multi-Path)

For **dynamic routing**, use:

```python
graph.add_conditional_edges("node_x", condition_function)
```

Where `condition_function(state)` returns the name of the next node:

```python
def route_logic(state):
    if state["intent"] == "math":
        return "math_solver"
    else:
        return "llm_answerer"
```

Example:

```python
graph.add_conditional_edges("router", route_logic)
```



### 🧭 Supported Transition Types

| Type        | Syntax                           | Use Case                    |
| ----------- | -------------------------------- | --------------------------- |
| Simple edge | `add_edge("a", "b")`             | Static flow                 |
| Conditional | `add_conditional_edges("x", fn)` | Dynamic branching (if/else) |
| End node    | `graph.set_entry_point("start")` | Set where graph begins      |
| Finish node | `graph.set_finish_point("end")`  | Set where graph ends        |



### ✅ Example

```python
graph.add_node("start", start_node)
graph.add_node("router", router_node)
graph.add_node("math_solver", solve_math)
graph.add_node("llm_answerer", answer_question)

graph.set_entry_point("start")
graph.set_finish_point("llm_answerer")

graph.add_edge("start", "router")

def route(state):
    return "math_solver" if state["task"] == "math" else "llm_answerer"

graph.add_conditional_edges("router", route)
```



### 📌 Summary

| Concept     | Description                       |
| ----------- | --------------------------------- |
| Edge        | A connection between nodes        |
| Purpose     | Defines execution path            |
| Conditional | Allows branching logic            |
| Static      | Always flows in a fixed direction |



#  Add conditional routing

### ✅ How to Add **Conditional Routing** in LangGraph

Conditional routing allows your graph to **branch dynamically** based on the **state**, similar to `if-else` logic in traditional programming.



### 🔁 What Is Conditional Routing?

It lets you **decide the next node** at runtime based on input, output, or other state variables.



### 🛠 Syntax for Conditional Routing

```python
graph.add_conditional_edges("current_node", condition_function)
```

* `"current_node"`: node after which routing decision happens
* `condition_function`: returns the **name of the next node**



### 📦 Step-by-Step Example

#### 1. Define Your Nodes

```python
def router_node(state):
    print("🔍 Deciding route...")
    return state

def math_node(state):
    print("🧮 Solving math...")
    return {"result": eval(state["input"])}

def llm_node(state):
    print("💬 Answering general question...")
    return {"response": "This is a general answer."}
```

#### 2. Define Routing Logic

```python
def route_logic(state):
    if "*" in state["input"] or "+" in state["input"]:
        return "math_node"
    else:
        return "llm_node"
```

#### 3. Build the Graph

```python
from langgraph.graph import StateGraph

builder = StateGraph()

# Add nodes
builder.add_node("router", router_node)
builder.add_node("math_node", math_node)
builder.add_node("llm_node", llm_node)

# Entry and exit
builder.set_entry_point("router")
builder.set_finish_point("llm_node")

# Conditional edge
builder.add_conditional_edges("router", route_logic)

graph = builder.compile()
```

#### 4. Run the Graph

```python
result = graph.invoke({"input": "5*7"})
print(result)
```



### ✅ Summary Table

| Step                    | Description                                 |
| ----------------------- | ------------------------------------------- |
| `add_conditional_edges` | Add edge that chooses path dynamically      |
| `condition_function`    | Determines the next node                    |
| Use Case                | Route LLM input to tools, agents, or chains |



# Setting up inputs/outputs

### 🔧 Setting Up Inputs and Outputs in **LangGraph**

In LangGraph, each **node** (function, chain, or tool) takes **inputs** and returns **outputs**, passed between steps using a **shared `state` dictionary**.



### 🧠 Concept

- ✅ **Input**: Data from the user or previous nodes  
- ✅ **Output**: Result added/updated in the `state` dict and passed forward



### 🗂 State Dictionary (Like Memory)

LangGraph passes a `dict` (called `state`) between nodes:

```python
state = {
    "input": "5*7",
    "result": "35"
}
```

You **read** values from it, and **write** outputs to it in every node.



### 🧱 Defining a Node with Inputs and Outputs

```python
def multiply_node(state: dict) -> dict:
    question = state["input"]
    a, b = map(int, question.split("*"))
    result = a * b
    return {"result": result}
```

This function:
- ✅ Reads `state["input"]`
- ✅ Adds `"result"` key with the output



### ✅ How Inputs/Outputs Are Chained

Example flow:

1. **User input node** adds `input` to state
2. **Math node** reads `input`, returns `result`
3. **Output node** prints or returns `result`



### 🛠 Example Graph

```python
from langgraph.graph import StateGraph

builder = StateGraph()

builder.add_node("get_input", lambda state: {"input": "5*7"})
builder.add_node("multiply", multiply_node)
builder.add_node("show_result", lambda state: print(f"Result is: {state['result']}"))

builder.set_entry_point("get_input")
builder.add_edge("get_input", "multiply")
builder.add_edge("multiply", "show_result")
builder.set_finish_point("show_result")

graph = builder.compile()
graph.invoke({})
```



### 🔁 Summary

| Element     | Description                              |
|-------------|------------------------------------------|
| `state`     | Dictionary passed between nodes          |
| Input       | Read from `state["key"]`                 |
| Output      | Return a dict with new/updated key-value |
| Chaining    | Outputs become inputs for the next node  |


# Running your graph with `.invoke()` and `.astream()`

### 🚀 Running Your LangGraph with `.invoke()` and `.astream()`

LangGraph offers two primary methods to execute your graph:



## 1. `.invoke()` – 🔁 **Synchronous Execution**

This is the **standard way** to run a LangGraph when all nodes execute synchronously.

### ✅ Usage

```python
output = graph.invoke({"input": "5*7"})
print(output)
```

### ✅ What it does:
- Runs the graph from start to finish
- Returns the **final state** after all nodes have completed
- Blocks execution until done

### ✅ Use when:
- You want simple execution
- You're not doing streaming or background processing



## 2. `.astream()` – ⚡ **Asynchronous Streaming Execution**

Use `.astream()` when you want to **stream outputs** from each node or observe the graph's progress in real-time.

### ✅ Usage

```python
import asyncio

async def run_stream():
    async for step in graph.astream({"input": "5*7"}):
        print("🧩 Step:", step)

asyncio.run(run_stream())
```

### ✅ What it does:
- Streams each intermediate state as the graph executes
- Useful for debugging, real-time feedback, or UIs

### ✅ Use when:
- You want progress updates (e.g., step-by-step agent tools)
- You want to show partial outputs or logs to the user



### ⚠️ Important

- `.invoke()` is **blocking and synchronous**
- `.astream()` is **non-blocking and async**, so use `asyncio.run()` if calling from a script



### 🔁 Summary

| Method     | Type        | Best Use Case                         |
|------------|-------------|----------------------------------------|
| `.invoke()` | Synchronous | Simple graphs, blocking execution     |
| `.astream()`| Async Stream| Real-time feedback, debugging, UI flow |
